In [1]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
from rnn_tools.model_saver import model_saver, model_saver_wandb
# import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data

c:\Users\Nick\Documents\GitHub\grvmodel\python


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [4]:
torch.__version__

'2.1.1'

In [5]:
['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']

['dist_last__1',
 'box_last__1',
 'speed_avg_1',
 'split_speed_v1_1',
 'split_speed_avg_1',
 'split_margin_avg_1',
 'margin_avg_1',
 'margin_time_avg_1',
 'RunHomeTime_1',
 'run_home_speed_1',
 'run_home_speed_v1_1',
 'first_out_avg_1',
 'pos_out_avg_1',
 'post_change_avg_1',
 'races_1',
 'wins_1',
 'wins_last_1',
 'weight_',
 'min_time_',
 'min_split_time_',
 'min_split_time_v1',
 'last_start_price',
 'last_start_prob']

In [6]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [7]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [8]:
tracks = ['Sandown Park',
 'The Meadows',
 'Warragul',
 'Geelong',
 'Sale',
 'Ballarat',
 'Bendigo',
 'Cranbourne']

In [9]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                
date = datetime.datetime.strptime("2020-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","SA","WA"]
states = ["VIC"]
states = ["NSW"]
# states = ["NZ"]
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
data_file = './data/gru_inputs_kitchen_sink_april.fth'
raceDB = rnn_tools.raceDB.build_dataset(data_file, hidden_size,date_filter=date,state_filter=states, margin_type='sftmin',v6=True,show_stats=False)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python
(1585482, 25)
106
Latest date = 2024-04-07 00:00:00
size after state filter (398315, 27)
(398315, 27)
Latest date = 2024-04-06 00:00:00
stats = [ 3.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  3.53000000e+01  8.00000000e+00  3.40000000e+02  4.00000000e+00
  1.70170170e+01 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  7.00000000e+00  5.71428571e-01
  1.00000000e+00  3.53000000e+01  1.99800000e+01 -1.00000000e+00
  1.50000000e+00  6.66666667e-01  4.07750000e+02  4.00000000e+00
  1.71422932e+01  1.72308717e+01  2.05000000e-01  3.43750000e+00
  2.45000000e-01  1.18050000e+01  1.73109945e+01  0.00000000e+00
  3.25000000e+00  7.50000000e-01  4.00000000e+00  5.00000000e-01
  2.00000000e+00  3.57000000e+01  2.30300000e+01  1.13000000e+01
  1.28000000e+00  3.33333333e-01  2.72000000e+02  3.00000000e+00
  1.69470405e+01  1.70203539e

100%|██████████| 25/25 [00:00<?, ?it/s]
c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\rnn_classes.py:419: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r.prob = torch.tensor((1/(torch.tensor(r.start_prices)+0.0001))/((1/torch.tensor(r.start_prices)).sum()+0.0001),device=self.device)


number of races = 54694, number of unique dogs = 18523
0           (Dapto, 297.0)
1        (Richmond, 330.0)
2           (Bulli, 400.0)
3          (Casino, 411.0)
4         (Lismore, 420.0)
               ...        
54689      (Casino, 484.0)
54690     (Lismore, 259.0)
54691     (Grafton, 350.0)
54692     (Lismore, 420.0)
54693     (Lismore, 420.0)
Length: 54694, dtype: object


In [10]:
topaz_data = pd.read_feather('./topaz/topaz_data.fth')

In [11]:
topaz_data.shape

(1346273, 326)

In [12]:
len([x for x in topaz_data.raceId.unique() if str(x) in raceDB.racesDict.keys()])

49965

In [13]:
raceDB.stats_cols = eval(raceDB.stats_cols)[0]

In [14]:
# raceDB.stats_cols = ['box']+raceDB.stats_cols

In [15]:
data = pd.read_feather('./data/gru_inputs_kitchen_sink_new_not_simple.fth')

In [16]:
data.columns

Index(['index', 'dogid', 'dog_name', 'raceid', 'race_grade', 'date',
       'race_time', 'race_num', 'trackOHE', 'track_name', 'state',
       'track_code', 'dist', 'bfSP', 'StartPrice', 'runtime', 'prev_race',
       'prev_race_date', 'prev_race_track', 'prev_race_state', 'place',
       'margin', 'box', 'stats', 'stats_cols'],
      dtype='object')

In [17]:
wins_only = data.query('place ==1')
wins_only['month'] = pd.to_datetime(wins_only['date']).dt.strftime('%Y-%m')
wins_only['year'] = pd.to_datetime(wins_only['date']).dt.strftime('%Y')

In [18]:
import plotly.graph_objects as go
import seaborn as sns
import plotly.figure_factory as ff
import pandas as pd
df = wins_only.dropna(subset=['bfSP'])
df = df.query('year == "2023"')


# Get the unique months
months = df['month'].unique()

# Create a list to hold the data for each month
hist_data = [df[df['month'] == month]['bfSP'].values for month in months]

# Create the figure
fig = ff.create_distplot(hist_data, months, show_hist=False)

# Update layout
fig.update_layout(
    title_text='Winning Price Distribution',
    xaxis_title_text='Winning Price',
    yaxis_title_text='Density',
    xaxis_range=[1,10]
)

fig.show()

In [19]:
for i,s in enumerate(raceDB.stats_cols):
    print(i,s)

0 inside
1 midfield
2 wide
3 weight
4 DogGrade
5 _dist_last__1
6 _box_last__1
7 _speed_avg_1
8 _split_speed_avg_1
9 _split_margin_avg_1
10 _margin_avg_1
11 _margin_time_avg_1
12 _RunHomeTime_1
13 _run_home_speed_1
14 _first_out_avg_1
15 _pos_out_avg_1
16 _post_change_avg_1
17 _races_1
18 _wins_1
19 _wins_last_1
20 _weight_
21 _min_time_
22 _min_split_time_
23 _last_start_price
24 _last_start_prob
25 dist_dist_last__10
26 dist_box_last__10
27 dist_speed_avg_10
28 dist_split_speed_avg_10
29 dist_split_margin_avg_10
30 dist_margin_avg_10
31 dist_margin_time_avg_10
32 dist_RunHomeTime_10
33 dist_run_home_speed_10
34 dist_first_out_avg_10
35 dist_pos_out_avg_10
36 dist_post_change_avg_10
37 dist_races_10
38 dist_wins_10
39 dist_wins_last_10
40 dist_weight_
41 dist_min_time_
42 dist_min_split_time_
43 dist_last_start_price
44 dist_last_start_prob
45 box_dist_last__10
46 box_box_last__10
47 box_speed_avg_10
48 box_split_speed_avg_10
49 box_split_margin_avg_10
50 box_margin_avg_10
51 box_margi

In [20]:
stat_list = ['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']


In [21]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 Reporting", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("val_ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      stat_mask = [1]*6+reg_stat_mask+[0]*80
      data_mask = [0]*26+[1]*20+[1]*20+[1]*20+[1]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask = [1]*6+reg_stat_mask+[0]*20+[0]*20+[0]*20+[0]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)
      input_size = sum(stat_mask)

      print(stat_mask)
      print(stat_mask.shape)


      if 'batch_days' in config.keys() and not raceDB.batches_setup:
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-07-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask,gen_packed_seq=True)
        raceDB.batches_setup = True
      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          # input_size = raceDB.batches['packed_x'][0].data[0].shape[0]



      print(f"{input_size=}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_extra(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      
      config['model_name'] = model.__class__.__name__
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        model.load_state_dict(model_data['model_state_dict'], strict=False)
        config['parent model'] = prev_model_file
        # raceDB.fill_hidden_states_dict_v2(model_data['db_train'])
        model = model.to(device)
        # raceDB.load_hidden_in_dict(hidden_in_dict=model_data['hidden_ins'],output_dict=model_data['output'])
        
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        # optimizer.load_state_dict(model_data['optim'])
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      wandb.run.name = f"{' '.join(states)}-{wandb.run.name}"
      model = model.to(device)
      print(model)

      # and use them to train the model
      # wandb.watch(model, log='all')
      try:
        return training_testing_gru_extra_data.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        # raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [22]:
raceDB.batches_setup = False

In [23]:
stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':1,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':1,
    '_pos_out_avg_1':1,
    '_post_change_avg_1':1,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':1,
    '_last_start_prob':1,
}

wandb_config_static = {
    'hidden_size': 256,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 2500,
    'learning_rate': 0.0001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2023-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 390,
    'stat_list_dict': stat_list_dict,
}

In [24]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [25]:
torch.cuda.empty_cache()

In [26]:
raceDB.race_prices_to_prob()

c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\rnn_classes.py:419: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [27]:
raceDB.latest_date

Timestamp('2024-04-06 00:00:00')

In [28]:
for race in raceDB.racesDict.values():
    race.loss = torch.tensor(1.0,requires_grad=True,device='cuda:0')  

In [29]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
# importlib.reload(training_testing_gru_extra_data_state_batch)
# importlib.reload(training_testing_gru_extra_data_toasty)
# importlib.reload(training_testing_gru_extra_data_toasty_og)
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='NZ-fine-waterfall-435', prev_model_version = 355)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# all_price_df = model_pipeline(raceDB,config=wandb_config_static,sweep=False)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 256, 'stats': ['inside', 'midfield', 'wide', 'weight', 'DogGrade', '_dist_last__1', '_box_last__1', '_speed_avg_1', '_split_speed_avg_1', '_split_margin_avg_1', '_margin_avg_1', '_margin_time_avg_1', '_RunHomeTime_1', '_run_home_speed_1', '_first_out_avg_1', '_pos_out_avg_1', '_post_change_avg_1', '_races_1', '_wins_1', '_wins_last_1', '_weight_', '_min_time_', '_min_split_time_', '_last_start_price', '_last_start_prob', 'dist_dist_last__10', 'dist_box_last__10', 'dist_speed_avg_10', 'dist_split_speed_avg_10', 'dist_split_margin_avg_10', 'dist_margin_avg_10', 'dist_margin_time_avg_10', 'dist_RunHomeTime_10', 'dist_run_home_speed_10', 'dist_first_out_avg_10', 'dist_pos_out_avg_10', 'dist_post_change_avg_10', 'dist_races_10', 'dist_wins_10', 'dist_wins_last_10', 'dist_weight_', 'dist_min_time_', 'dist_min_split_time_', 'dist_last_start_price', 'dist_last_start_prob', 'box_dist_last__10', 'box_box_last__10', 'box_speed_avg_10', 'box_split_speed_avg_10', 'box_split_margin_a

100%|██████████| 18523/18523 [00:08<00:00, 2274.30it/s]


[(datetime.date(2019, 12, 1), datetime.date(2020, 12, 25)), (datetime.date(2020, 12, 25), datetime.date(2022, 1, 19)), (datetime.date(2022, 1, 19), datetime.date(2022, 12, 31))]
2020-12-25
2022-01-19
2022-12-31
Train examples [11238, 13382, 12513]


100%|██████████| 18522/18522 [00:21<00:00, 845.37it/s]
3it [01:05, 21.71s/it]


...

  2%|▏         | 50/2500 [42:00<30:54:59, 45.43s/it]

New Max ROI: -0.03028942243803751, -0.009127371137187942, 2.08193302154541


  2%|▏         | 55/2500 [46:16<31:09:26, 45.88s/it]

New Max ROI: -0.030925270066938234, -0.008604998317027355, 2.077793836593628


  2%|▏         | 62/2500 [52:36<35:53:46, 53.01s/it]

In [ ]:
profit_model = rnn_classes.GRUNetv3_profit(raceDB).to('cuda:0')

In [ ]:
raceDB.states

In [ ]:
new_batches = []
for batch in batch_races:
    batch_dict = {state:[] for state in raceDB.states}
    for race in batch:
        batch_dict[race.state].append(race)
    new_batch = list(batch_dict.values())
    new_batches.append(new_batch)
    for k,v in batch_dict.items():
        print(f"{k} races {len(v)}")
raceDB.batches['new_batch_races'] = new_batches

In [ ]:
[print(len(batch)) for batch in raceDB.batches['new_batch_races']]

In [ ]:
list(batch_dict.values())

In [ ]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

# profit_tensor =  model_pipeline(raceDB,config=wandb_config_static,sweep=False)

In [ ]:
def print_grad_fn(tensor):
    print(tensor)
    if tensor is not None:
        if tensor is not None:
            for t in tensor.next_functions:
                if t[0] is not None:
                    # print(f"{t[0]=}")
                    print_grad_fn(t[0])

# Usage:
# Assuming `output` is your tensor
# print_grad_fn(output)

In [ ]:
profit_tensor.grad_fn.next_functions

In [ ]:
profit_tensor.shape

In [ ]:
print_grad_fn(profit_tensor[0].grad_fn)

In [ ]:
profit_tensor.grad_fn.next_functions

In [ ]:
len_test = len(raceDB.test_race_ids)
test_idx = range(0,len_test)
race = raceDB.get_test_input(test_idx)

In [ ]:
for r in race:
    print(r.prices)
    print(r.raceid)
    _, actual = torch.max(r.classes, 0)
    onehot_win = F.one_hot(actual, num_classes=8)
    print(onehot_win)
    print(r.win_price_weight)
    print(r.raw_margins)
    print(r.race_date)
    print(r.track_name)
    print('----------------------------')

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp

hidden_outs = []
margins = []
for race in raceDB.dogsDict['410839665'].races.values():
    hidden_outs.append(race.hidden_out.detach())
    margins.append(race.margin)
# Assuming hidden_states is your list of tensors
hidden_states = [x[0:256] for x in hidden_outs]
hidden_states = [hidden_state.reshape(16, 16) for hidden_state in hidden_states]
# Create a heatmap for each step
frames = [go.Frame(
    data=[go.Heatmap(z=hidden_state.tolist(), colorscale='Viridis')],
    name=str(i)
) for i, hidden_state in enumerate(hidden_states)]

# Create a slider
sliders = [dict(
    steps=[dict(method='animate',
                args=[[frame['name']]],
                label=str(i)) for i, frame in enumerate(frames)],
    transition=dict(duration=300, easing='cubic-in-out'),
    active=0,
)]

# Create a layout
layout = go.Layout(
    sliders=sliders,
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, {"frame": {"duration": 500, "redraw": False},
                                                 "fromcurrent": True,
                                                 "transition": {"duration": 300,
                                                                "easing": "quadratic-in-out"}}]),
                                  dict(label='Stop',
                                       method='animate',
                                       args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}])])],
)

# Create a figure
fig = go.Figure(data=frames[0]['data'], layout=layout, frames=frames)

# Display the figure
fig.show()

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
hidden_inits= [hidden_state[1,:].reshape(16, 16) for hidden_state in raceDB.hidden_state_inits]

dogs = ['366590730','410839665']

hiddens = []
margins = []
for dog in dogs:
    hidden_outs = []
    margin = []
    for race in raceDB.dogsDict[dog].races.values():

        hidden_outs.append(race.hidden_out.detach())
        margin.append(race.margin)
    hiddens.append(hidden_outs)
    margins.append(margin)
    
min_len = min([len(x) for x in hiddens])
print(f"{min_len=}")
hidden_states = [[hidden_state[0:256].cpu().numpy().reshape(16, 16) for hidden_state in dog[0:min_len]] for dog in hiddens]
print(f"{[len(x) for x in hidden_states]=}")
# Create a heatmap for each step
global_min = min(hidden_state.min() for hidden_state in hidden_states[0])
global_max = max(hidden_state.max() for hidden_state in hidden_states[0])

print(global_min, global_max)
# print(hidden_states[0])
# Create a heatmap for each step with constant scale
# Create a subplot for each dog
# Create a subplot for each dog with an additional row for the line plot
fig = make_subplots(rows=4, cols=1)

# Create a heatmap and line plot for each dog
for i, dog_hidden_states in enumerate(hidden_states):
    # Create heatmap
    heatmap = go.Heatmap(z=dog_hidden_states[0].tolist(), zmin=global_min, zmax=global_max, colorscale='Viridis')
    fig.add_trace(heatmap, row=2*i+2, col=1)

    # Create line plot
    line_plot = go.Scatter(y=[sum(abs(state.flatten())) for state in dog_hidden_states], mode='lines',showlegend=False)
    fig.add_trace(line_plot, row=2*i+1, col=1)

frames = []

# Iterate over dogs and their corresponding hidden states
for j in range(min_len):
    frame_data = []
    for i, dog_hidden_states in enumerate(hidden_states):
        hidden_state = dog_hidden_states[j]
        heatmap = go.Heatmap(z=hidden_state.tolist(), zmin=global_min, zmax=global_max, colorscale='Viridis')
        line_plot = go.Scatter(y=[sum(abs(state.flatten())) for state in dog_hidden_states[:j+1]], mode='lines',showlegend=False)
        frame_data.extend([heatmap, line_plot])
    frame = go.Frame(data=frame_data, name=str(j))  # Create a frame for each time step
    frames.append(frame)

# Define the slider
# Define the slider
sliders = [dict(steps=[dict(method='animate',
                            args=[[f.name], {"mode": "immediate",
                                             "frame": {"duration": 100, "redraw": True},
                                             "transition": {"duration": 0}}],
                            label=f.name) for f in frames],
                active=0)]

# Update layout
fig.update_layout(
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, {"frame": {"duration": 100, "redraw": False},
                                                 "fromcurrent": True,
                                                 "mode": "immediate",
                                                 "transition": {"duration": 0,
                                                                "easing": "quadratic-in-out"}}]),
                                  dict(label='Stop',
                                       method='animate',
                                       args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}])])],
    sliders=sliders
)
# Update frames
fig.frames = frames

# Display the figure
fig.show()
import plotly.io as pio

# Save the figure as an interactive html file
pio.write_html(fig, 'hidden_simple_double.html')

In [ ]:
# Create a line plot for each dog
fig = go.Figure()

for i, dog_hidden_states in enumerate(hidden_states):
    # Calculate the absolute sum of hidden states for each time step
    abs_sum = [sum(abs(state.flatten())) for state in dog_hidden_states]
    print(abs_sum)

    # Create line plot with x values specified
    line_plot = go.Scatter(x=list(range(len(dog_hidden_states))), y=abs_sum, mode='lines', name=f'Dog {i+1}')
    fig.add_trace(line_plot)

# Update layout
fig.update_layout(
    title="Absolute Sum of Hidden States Over Time",
    xaxis_title="Time Step",
    yaxis_title="Absolute Sum of Hidden States",
    legend_title="Dogs",
)

# Display the figure
fig.show()

In [ ]:
from bokeh.layouts import column
from bokeh.models import Slider, CustomJS
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import curdoc
import numpy as np

# Assuming hidden_states is a list of 2D numpy arrays
hidden_states = [[hidden_state[0:256].cpu().numpy().reshape(16, 16) for hidden_state in dog[0:min_len]] for dog in hiddens]

# Create a ColumnDataSource to hold the data for the current frame
source = ColumnDataSource(data={'image': [hidden_states[0][0]]})

# Create a figure and add an image renderer
p = figure(x_range=(0, 1), y_range=(0, 1), width=400, height=400, match_aspect=True)
p.image(image='image', x=0, y=0, dw=1, dh=1, source=source)

# Create a slider that goes from the first to the last frame
slider = Slider(start=0, end=len(hidden_states[0])-1, value=0, step=1)

# Define a callback function to update the ColumnDataSource's data when the slider is changed
callback = CustomJS(args=dict(source=source, states=hidden_states[0]), code="""
    var i = cb_obj.value;
    source.data = {'image': [states[i]]};
""")

# Attach the callback to the slider
slider.js_on_change('value', callback)

# Display the figure and slider
layout = column(p, slider)
show(layout)

In [ ]:
for i in raceDB.hidden_state_inits:
    print(i[0,0])

In [ ]:
hidden_state_init = model.h0
raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=256)

In [ ]:
raceDB.dogsDict['408750377'].hidden